In [10]:
from pyspark.ml.feature import VectorAssembler
from pyspark.sql import Row
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.classification import LogisticRegressionWithLBFGS
from pyspark.mllib.tree import GradientBoostedTrees
from pyspark.mllib.tree import GradientBoostedTreesModel
from pyspark.ml.linalg import Vectors

In [2]:
ctx = SparkSession.builder \
    .master("spark://hs:7077") \
    .appName("bosch spark") \
    .getOrCreate()
file_dir = "/home/h/kaggle_fun/bosch/raw_data/"

** - 1 change dataframe to vector ** 
      [labeled point](http://stackoverflow.com/questions/32556178/create-labeledpoints-from-spark-dataframe-in-python)
      [比较全的doc](http://www.techpoweredmath.com/spark-dataframes-mllib-tutorial/)    
** - 2 missing values count **   
** - 3 fill missing values **   
** - 4 plot feature point distribution **  
** - 5 normalize the feature **  
** - 6 adjust positive negative ratio **  
** - 7 train model **    
** - 8 evaluate model matric **   
** - 9 [feature engineer](http://stackoverflow.com/questions/32982425/encode-and-assemble-multiple-features-in-pyspark) **   
** -10 [spark python vs scala](http://stackoverflow.com/questions/32464122/spark-performance-for-scala-vs-python) **  
sadly finished already!



In [3]:
def load_csv_df(file_name):
    return ctx.read.csv(file_dir + file_name, header=True, inferSchema=True)

def check_p_n_cnt(input_df, col_name, pos):
    tmp_table_name = "check_p_n_input_df"
    input_df.registerTempTable(tmp_table_name)
    total = ctx.sql("select count(*) as tcnt from %s" % tmp_table_name)
    pos = ctx.sql("select count(*) as pcnt from %s where %s = %s" %(tmp_table_name, col_name, pos))
    print "positive/total : %s/%s" % (pos.collect()[0]["pcnt"], total.collect()[0]["tcnt"])

def adjust_p_n_ratio(input_df, cate_col_name, scale, raw_ratio):
    tmp_table_name = "adjust_p_n_input_df"
    input_df.registerTempTable(tmp_table_name)
    adjusted_df = ctx.sql("""select * from %s where %s = 1 or (%s = 0 and rand() < %s*%s)"""
                          % (tmp_table_name, cate_col_name, cate_col_name, scale, raw_ratio))
    return adjusted_df

def df_to_vec(input_df, ignore_list=[]):
    assembler = VectorAssembler(
        inputCols=[x for x in input_df.columns if x not in ignore_list],
        outputCol='features')
    return assembler.transform(input_df)

def df_to_vec_2(input_df, ignore_list = []):
    labeled_points = input_df.rdd.map(lambda line:LabeledPoint(int(line[-1]),line[0:-1]))
    return labeled_points

def get_corr_matrix(vec, method="pearson"):
    return vec.stat.corr(vec)

def train_lr_model(input_vec):
    lrm = LogisticRegressionWithSGD.train(input_vec, iterations=10)
    return lrm

def train_gbdt_model(input_vec):
    gbdt_model = GradientBoostedTrees.trainClassifier(input_vec,{}, numIterations = 100)
    return gbdt_model

In [4]:
train_numer_df = load_csv_df("train_numeric.csv")
test_numer_df = load_csv_df("test_numeric.csv")

In [5]:
#check_p_n_cnt(train_numer_df, "Response", "1") #6879/1183747
adjusted_train_num = adjust_p_n_ratio(train_numer_df, "Response", 1.2, float(6879)/1183747)
print "pos", adjusted_train_num.filter(adjusted_train_num.Response == 1).count()
print "neg", adjusted_train_num.filter(adjusted_train_num.Response == 0).count()

pos 6879
neg 8107


In [9]:
#train the model
train_numer_vec = df_to_vec_2(adjusted_train_num)
gbdt_model = train_gbdt_model(train_numer_vec)
#print gbdt_model.save(sc, "gbdt_model")

In [7]:
test_numer_vec = test_numer_df.rdd.map(lambda data: Vectors.dense([c for c in data]))

In [12]:
gbdt_model = GradientBoostedTreesModel.load(sc, "gbdt_model")
predict_res = gbdt_model.predict(test_numer_vec)

In [22]:
gbdt_model.predict(test_numer_vec.take(20000))

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 17.0 failed 1 times, most recent failure: Lost task 0.0 in stage 17.0 (TID 187, localhost): org.apache.spark.SparkException: Kryo serialization failed: Buffer overflow. Available: 0, required: 103298. To avoid this, increase spark.kryoserializer.buffer.max value.
	at org.apache.spark.serializer.KryoSerializerInstance.serialize(KryoSerializer.scala:300)
	at org.apache.spark.executor.Executor$TaskRunner.run(Executor.scala:313)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1142)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:617)
	at java.lang.Thread.run(Thread.java:745)

Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:1450)
	at org.apache.spark.sc

In [79]:
ls gbdt_model/data/

part-r-00000-d3f2cc46-81b9-4e2c-acdc-37a565f078c7.snappy.parquet
part-r-00001-d3f2cc46-81b9-4e2c-acdc-37a565f078c7.snappy.parquet
part-r-00002-d3f2cc46-81b9-4e2c-acdc-37a565f078c7.snappy.parquet
part-r-00003-d3f2cc46-81b9-4e2c-acdc-37a565f078c7.snappy.parquet
part-r-00004-d3f2cc46-81b9-4e2c-acdc-37a565f078c7.snappy.parquet
part-r-00005-d3f2cc46-81b9-4e2c-acdc-37a565f078c7.snappy.parquet
part-r-00006-d3f2cc46-81b9-4e2c-acdc-37a565f078c7.snappy.parquet
part-r-00007-d3f2cc46-81b9-4e2c-acdc-37a565f078c7.snappy.parquet
_SUCCESS


In [19]:
#train_numer_vec = df_to_vec(train_numer_df, ["Id"])
#train_numer_vec.corr(method="pearson")
ignore_list = ["Response"]
assembler = VectorAssembler(
        inputCols=[x for x in train_numer_df.columns if x not in ignore_list],
        outputCol='features')
#print type(train_numer_vec)
train_numer_vec = assembler.transform(train_numer_df)